In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

import geopandas as gpd
from shapely.ops import nearest_points

import warnings
warnings.filterwarnings("ignore")

import os
GEODATA_PATH = 'data/geodata/'
TABULAR_PATH = 'data/tabular/'
OUTPUT_PATH = 'data/output/'
data_files = os.listdir(TABULAR_PATH)

Helper Functions

In [2]:
def get_sex_age_cols(pop_df,sex="bot",min_age=7,max_age=25):
    clean_cols = [col.replace('und_','').replace('_and_ove','') for col in pop_df.columns]
    pop_df.columns = clean_cols
    sex_single_age_cols = [col for col in clean_cols if col.split("_")[-2:][0]==sex]
    age_groups = [col for col in sex_single_age_cols if (int(col.split("_")[-1])>=min_age)\
                        & (int(col.split("_")[-1])<=max_age)]
    return age_groups

def get_road_brgy_psgc(road_geom,brgy_df):
    brgy_intersects  = brgy_df['geometry'].intersects(road_geom)
    brgy_codes = set(brgy_df.loc[brgy_intersects,"Bgy_Code"].values.tolist())

    return brgy_codes

def get_road_prov_info(road_geom,brgy_df):
    brgy_intersects  = brgy_df['geometry'].intersects(road_geom)
    prov_names = set(brgy_df.loc[brgy_intersects,"Pro_Name"].values)

    return prov_names

Schools data

In [3]:
population_df = pd.read_excel(TABULAR_PATH+'Population 2010, 2015 ARMM,BCT.xlsx')
cond = population_df['PSGC_REGI']=='PH150000000'
population_df=population_df[cond].reset_index(drop=True)
print("population_df",population_df.shape)

schools_meta_df = pd.read_excel(TABULAR_PATH+'SchoolsDepEd2017.xlsx',
                                usecols=['SCH_ID', 'SCH_NAME', 'SCH_CAT', 'SCH_TYPE', 
                                         'LAT', 'LONG', 'BARANGAY','MUNNAME', 'PROV'])
schools_meta_df["SCH_TYPE"] = schools_meta_df["SCH_TYPE"].str.upper()
print("schools_meta_df",schools_meta_df.shape)

population_df['brgy_muni_prov_key']=population_df["BrgyName"].str.replace("Pagatin \(Pagatin I\)","Pagatin I")\
                                                            .str.split(" \(").str[0].str.strip()\
                                    +", "+population_df["MuniCities"].str.split(" \(").str[0].str.strip()\
                                    +", "+population_df["PROV"].str.split(" \(").str[0].str.strip()
population_df['brgy_muni_prov_key']=population_df['brgy_muni_prov_key'].str.lower().str.strip()

schools_meta_df['brgy_muni_prov_key'] = schools_meta_df["BARANGAY"].str.split(" \(").str[0].str.strip()\
                                        +", "+schools_meta_df["MUNNAME"].str.split(" \(").str[0].str.strip()\
                                        +", "+schools_meta_df["PROV"].str.split(" \(").str[0].str.strip()
schools_meta_df['brgy_muni_prov_key'] = schools_meta_df['brgy_muni_prov_key'].str.lower()\
                                        .str.replace("city of marawi","marawi city")\
                                        .str.replace("lamitan city","city of lamitan")
schools_meta_update = pd.merge(schools_meta_df,population_df[["PSGC_BRGY","brgy_muni_prov_key"]],
                               on="brgy_muni_prov_key",how="left")

schools_meta_update.drop(columns="brgy_muni_prov_key",inplace=True)
print("schools_meta_update",schools_meta_update.shape)

manual_clean = {
    133036:'PH150703024',
    133047:'PH150703025',
    133333:'PH153634057',
    134011:'PH153807005',
    216650:'PH153835007',
    408573:'PH153819001',
    133539:'PH153637005',
    133540:'PH153637006',
    304962:'PH153637006',
    133541:'PH153637012',
    133542:'PH153637016',
    133543:'PH153637018',
    133544:'PH153637021',
    133784:'PH153632019',
    134158:'PH153812001',
    216613:'PH153819021',
    133833:'PH153635052',
    318215:'PH153635016',
    324703:'PH153635042',
    324730:'PH153635042',
    304993:'PH153635029'
}

for school_id,psgc_brgy in manual_clean.items():
    cond = schools_meta_update["SCH_ID"]==school_id
    schools_meta_update.loc[cond,"PSGC_BRGY"]=psgc_brgy
    
schools_meta_update = pd.merge(schools_meta_update,population_df[["PSGC_BRGY","PSGC_PROV","PSGC_CITY/MUNI"]],
                               on="PSGC_BRGY",how="left")
print("schools_meta_update",schools_meta_update.shape)

schools_meta_update.to_csv(OUTPUT_PATH+"SchoolsDepEd2017_Clean.csv",index=False)

missing_count = sum(schools_meta_update["PSGC_BRGY"].isna())
print("\nmissing in brgy_muni_prov_key: ",missing_count)

population_df (2482, 288)
schools_meta_df (2711, 9)
schools_meta_update (2711, 10)
schools_meta_update (2711, 12)

missing in brgy_muni_prov_key:  9


In [4]:
schools_clean = pd.read_csv(OUTPUT_PATH+"SchoolsDepEd2017_Clean.csv",usecols=["SCH_ID","SCH_NAME","SCH_TYPE","SCH_CAT","PSGC_BRGY"])
cond = (schools_clean["SCH_TYPE"]=="PUBLIC") & (schools_clean["SCH_CAT"].str.contains("ES|PS"))
school_public_elem = schools_clean[cond].reset_index(drop=True)
school_public_elem = school_public_elem.drop(columns=["SCH_TYPE","SCH_CAT"])
print("school_public_elem",school_public_elem.shape)

schools_elementary_enrollment = pd.read_csv(TABULAR_PATH+"deped_publicelementaryenrollment2015.csv",
                                           encoding='latin',
                                           usecols=["school_id","region","school_name","year_level","gender","enrollment"])
cond = schools_elementary_enrollment["region"]=="ARMM - Autonomous Region in Muslim Mindanao"
schools_elementary_enrollment = schools_elementary_enrollment[cond].reset_index(drop=True)
print("schools_ementary_enrollment",schools_elementary_enrollment.shape,
      "unique schools:", len(schools_elementary_enrollment["school_id"].unique()))

schools_elementary_enrollment = pd.pivot_table(schools_elementary_enrollment,index=["school_id"],
                                              columns=["gender","year_level"],values="enrollment")
schools_elementary_enrollment = schools_elementary_enrollment.reset_index()

new_cols = [gender+'_'+year_level for gender,year_level in schools_elementary_enrollment.columns.to_flat_index()]
schools_elementary_enrollment.columns = ["school_id"]+new_cols[1:]

schools_elementary_enrollment = pd.merge(schools_elementary_enrollment,school_public_elem,left_on="school_id",right_on="SCH_ID",how="left")
schools_elementary_enrollment.drop(columns="SCH_ID",inplace=True)

print("schools_elementary_enrollment",schools_elementary_enrollment.shape)

schools_elementary_enrollment.to_csv(OUTPUT_PATH+"public_elementary_enrollment2015_Clean.csv",index=False)

print("\nmissing school id:",list(schools_elementary_enrollment.loc[schools_elementary_enrollment["SCH_NAME"].isna(),"school_id"].unique()))

school_public_elem (2101, 3)
schools_ementary_enrollment (6036, 6) unique schools: 503
schools_elementary_enrollment (503, 15)

missing school id: [133004]


In [5]:
cond = (schools_clean["SCH_TYPE"]=="PUBLIC") & (schools_clean["SCH_CAT"].str.contains("HS|PS"))
school_public_secondary = schools_clean[cond].reset_index(drop=True)
school_public_secondary = school_public_secondary.drop(columns=["SCH_TYPE","SCH_CAT"])
print("school_public_secondary",school_public_secondary.shape)

schools_secondary_enrollment = pd.read_csv(TABULAR_PATH+"deped_publicsecondaryenrollment2015.csv",
                                           encoding='latin',
                                           usecols=["school_id","region","school_name","year_level","gender","enrollment"])
cond = schools_secondary_enrollment["region"]=="ARMM - Autonomous Region in Muslim Mindanao"
schools_secondary_enrollment = schools_secondary_enrollment[cond].reset_index(drop=True)
print("schools_secondary_enrollment",schools_secondary_enrollment.shape,
      "unique schools:", len(schools_secondary_enrollment["school_id"].unique()))

schools_secondary_enrollment = pd.pivot_table(schools_secondary_enrollment,index=["school_id"],
                                              columns=["gender","year_level"],values="enrollment")
schools_secondary_enrollment = schools_secondary_enrollment.reset_index()

new_cols = [gender+'_'+year_level for gender,year_level in schools_secondary_enrollment.columns.to_flat_index()]
schools_secondary_enrollment.columns = ["school_id"]+new_cols[1:]

schools_secondary_enrollment = pd.merge(schools_secondary_enrollment,school_public_secondary,left_on="school_id",right_on="SCH_ID",how="left")
schools_secondary_enrollment.drop(columns="SCH_ID",inplace=True)

print("schools_secondary_enrollment",schools_secondary_enrollment.shape)

schools_secondary_enrollment.to_csv(OUTPUT_PATH+"public_secondary_enrollment2015_Clean.csv",index=False)

print("\nmissing school id:",list(schools_secondary_enrollment.loc[schools_secondary_enrollment["SCH_NAME"].isna(),"school_id"].unique()))

school_public_secondary (956, 3)
schools_secondary_enrollment (2432, 6) unique schools: 304
schools_secondary_enrollment (304, 11)

missing school id: [326502]


GeoData

In [6]:
brgys_geodata = gpd.read_file(GEODATA_PATH+'AdministrativeBoundariesBARMMBarangays20190206PSA2016/AdministrativeBoundariesBARMMBarangays20190206PSA2016.shp')
cond = brgys_geodata['Reg_Code']=='PH150000000'
brgys_geodata=brgys_geodata[cond].reset_index(drop=True)
brgys_geodata = brgys_geodata.to_crs("EPSG:4326")
print("brgys_geodata",brgys_geodata.shape)

schools_geodata = gpd.read_file(GEODATA_PATH+'SchoolsARMMDEPED2017/SchoolsARMMDEPED2017.shp')
schools_meta_update = pd.read_csv(OUTPUT_PATH+"SchoolsDepEd2017_Clean.csv")
schools_geodata_update = gpd.GeoDataFrame(schools_meta_update, geometry = schools_geodata["geometry"])
schools_geodata_update.to_file(OUTPUT_PATH+"SchoolsDepEd2017_Geodata_Clean/SchoolsDepEd2017_Geodata_Clean.shp")
print("schools_geodata_update",schools_geodata_update.shape)
schools_geodata = schools_geodata[["SCH_ID","geometry"]]
print("schools_geodata",schools_geodata.shape)

brgys_geodata (2489, 9)
schools_geodata_update (2711, 13)
schools_geodata (2711, 2)


Population Data

In [7]:
population_df = pd.read_excel(TABULAR_PATH+'Population 2010, 2015 ARMM,BCT.xlsx')
cond = population_df['PSGC_REGI']=='PH150000000'
population_df=population_df[cond].reset_index(drop=True)
print("population_df",population_df.shape)

population_age_groups_df = population_df[["PSGC_BRGY"]]

custom_age_ranges = [(6,12),(13,16),(17,18),(19,22),(6,16),(0,84)]

age_range=5
popn_age_ranges = [(age,age+age_range-1) for age in range(0,81,age_range)] + custom_age_ranges

for min_age,max_age in popn_age_ranges:

    both_age_cols = get_sex_age_cols(population_df,sex="bot",min_age=min_age,max_age=max_age)
    male_age_cols = get_sex_age_cols(population_df,sex="mal",min_age=min_age,max_age=max_age)
    female_age_cols = get_sex_age_cols(population_df,sex="fem",min_age=min_age,max_age=max_age)
    
    both_col = 'both_age_{}_to_{}'.format(min_age,max_age)
    male_col = 'male_age_{}_to_{}'.format(min_age,max_age)
    female_col = 'female_age_{}_to_{}'.format(min_age,max_age)
    
    age_groups = population_df[both_age_cols+male_age_cols+female_age_cols]
    age_groups[both_col] = age_groups.loc[:,both_age_cols].sum(axis=1)
    age_groups[male_col] = age_groups.loc[:,male_age_cols].sum(axis=1)
    age_groups[female_col] = age_groups.loc[:,female_age_cols].sum(axis=1)

    age_groups.drop(columns=both_age_cols+male_age_cols+female_age_cols,inplace=True)
    
    population_age_groups_df = pd.concat([population_age_groups_df,age_groups],axis=1)
    del age_groups

total_popn_cols = {
    "both_age_0_to_84":"total_popn",
    "male_age_0_to_84":"total_male_popn",
    "female_age_0_to_84":"total_female_popn"
}
population_age_groups_df.rename(columns=total_popn_cols,inplace=True)    
print("population_age_groups_df",population_age_groups_df.shape)

population_df (2482, 288)
population_age_groups_df (2482, 70)


Schools

In [8]:
schools_df = pd.read_excel(TABULAR_PATH+'Number of schools per brgy.xlsx',usecols=["PSGC_BRGY","Number of School"])
schools_df.rename(columns={'Number of School':'No_of_Schools'},inplace=True)
print("schools_df",schools_df.shape)

schools_meta_df = pd.read_csv(OUTPUT_PATH+'SchoolsDepEd2017_Clean.csv',usecols=["PSGC_BRGY","SCH_CAT","SCH_TYPE"])
schools_meta_df["SCH_TYPE"] = schools_meta_df["SCH_TYPE"].str.strip().str.upper()
schools_meta_df = pd.get_dummies(schools_meta_df, columns=["SCH_TYPE","SCH_CAT"])
schools_meta_df = schools_meta_df.groupby("PSGC_BRGY",as_index=False).sum()
print('schools_meta_df',schools_meta_df.shape)

education_df = pd.merge(population_age_groups_df,schools_df,on="PSGC_BRGY",how="left")
education_df = pd.merge(education_df,schools_meta_df,on="PSGC_BRGY",how="left")
print("education_df",education_df.shape)

schools_df (1570, 2)
schools_meta_df (1758, 14)
education_df (2482, 84)


SDG-related

In [9]:
source_light_df = pd.read_excel(TABULAR_PATH+'Number of Households by Kind of Fuel for Lighting (ARMM 2015) Brgy.xlsx')
source_light_df["SDG_Households_Electricity_Light_Pct"] = \
            source_light_df["Electricity"]/source_light_df["Number of HH"]
source_light_df["SDG_Households_Kerosene_Gass_Light_Pct"] = \
            source_light_df["Kerosene (Gaas)"]/source_light_df["Number of HH"]
source_light_df = source_light_df[["PSGC","SDG_Households_Electricity_Light_Pct","SDG_Households_Kerosene_Gass_Light_Pct"]]
print("source_light_df",source_light_df.shape)

source_drinking_water_df = pd.read_excel(TABULAR_PATH+'Main Source of Water Supply for Drinking (ARMM 2015) Brgy.xlsx')
source_drinking_water_df["SDG_Households_Drinking_Owned_Faucet_Pct"] = \
            source_drinking_water_df["Own use faucet community water system"]/source_drinking_water_df["Number of Households"]
source_drinking_water_df["SDG_Households_Drinking_Shared_Faucet_Pct"] = \
            source_drinking_water_df["Shared faucet community water system"]/source_drinking_water_df["Number of Households"]
source_drinking_water_df = source_drinking_water_df[["PSGC","SDG_Households_Drinking_Owned_Faucet_Pct","SDG_Households_Drinking_Shared_Faucet_Pct"]]
print("source_drinking_water_df",source_drinking_water_df.shape)

construction_materials_df = pd.read_excel(TABULAR_PATH+'Construction Materials of the Walls(ARMM 2015)Brgy.xlsx')
construction_materials_df["SDG_Households_Construction_Material_Concrete_Pct"] = \
           construction_materials_df["Concrete/brick/stone"]/construction_materials_df["Total Occupied Housing Units"] 
construction_materials_df = construction_materials_df[["PSGC","SDG_Households_Construction_Material_Concrete_Pct"]]
print("construction_materials_df",construction_materials_df.shape)

building_tenure_df = pd.read_excel(TABULAR_PATH+'TypeofBuildingTenureStatusoftheHousingUnitLot(ARMM 2015)Brgy.xlsx')
building_tenure_df["SDG_Building_Tenure_Type_Owned_Pct"] = \
            building_tenure_df["Own or owner like possession of house and lot"]/building_tenure_df["Number of HH"]
building_tenure_df = building_tenure_df[["PSGC", "SDG_Building_Tenure_Type_Owned_Pct"]]
print("building_tenure_df",building_tenure_df.shape)

type_bulding_df = pd.read_excel(TABULAR_PATH+'Type of Building (ARMM 2015) Brgy.xlsx')
type_bulding_df["SDG_Type_Building_Single_Pct"] = \
            type_bulding_df["Single house"]/type_bulding_df["Total Occupied Housing Units"]
type_bulding_df = type_bulding_df[["PSGC","SDG_Type_Building_Single_Pct"]]
print("type_bulding_df",type_bulding_df.shape)

gainful_workers_df = pd.read_excel(TABULAR_PATH+'Gainful Workers BRGY.xlsx')
gainful_workers_df['SDG_Type_Worker_15_to_24'] = \
            (gainful_workers_df['tot_15 - 19_both_sexes']+gainful_workers_df['tot_20 - 24_both_sexes'])/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Man'] = \
            gainful_workers_df['man_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Pro'] = \
            gainful_workers_df['pro_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Tec'] = \
            gainful_workers_df['tec_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Cle'] = \
            gainful_workers_df['cle_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Ser'] = \
            gainful_workers_df['ser_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Ski'] = \
            gainful_workers_df['ski_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Cra'] = \
            gainful_workers_df['cra_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Pla'] = \
            gainful_workers_df['pla_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Ele'] = \
            gainful_workers_df['ele_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df['SDG_Type_Worker_Arm'] = \
            gainful_workers_df['arm_15_ovr_both_sexes']/gainful_workers_df['tot_15_ovr_both_sexes']
gainful_workers_df = gainful_workers_df[['PSGC', 'SDG_Type_Worker_Man', 'SDG_Type_Worker_Pro', 'SDG_Type_Worker_Tec', 'SDG_Type_Worker_Cle',
                       'SDG_Type_Worker_Ser', 'SDG_Type_Worker_Ski', 'SDG_Type_Worker_Cra', 'SDG_Type_Worker_Pla',
                       'SDG_Type_Worker_Ele', 'SDG_Type_Worker_Arm']]
print("gainful_workers_df",gainful_workers_df.shape)

floor_area_df = pd.read_excel(TABULAR_PATH+'Floor Area (ARMM, 2010) Brgy.xlsx')
floor_area_df['SDG_less_than_54_sqft'] = \
            floor_area_df['Less than 5 sq.m./less than 54 sq.ft.']/floor_area_df['Total Occupied Housing Units']
floor_area_df = floor_area_df[['PSGC', 'SDG_less_than_54_sqft']]
print("floor_area_df",floor_area_df.shape)

dataframes = [source_light_df,source_drinking_water_df,construction_materials_df,building_tenure_df,type_bulding_df, 
              gainful_workers_df, floor_area_df]

sdg_df = reduce(lambda  left,right: pd.merge(left,right,on=['PSGC'],how='left'), dataframes)
print("sdg_df",sdg_df.shape)

education_sdg_df = pd.merge(education_df,sdg_df,left_on="PSGC_BRGY",right_on="PSGC",how="left")
education_sdg_df.drop(columns="PSGC",inplace=True)
print("education_sdg_df",education_sdg_df.shape)

source_light_df (2490, 3)
source_drinking_water_df (2490, 3)
construction_materials_df (2490, 2)
building_tenure_df (2490, 2)
type_bulding_df (2490, 2)
gainful_workers_df (2490, 11)
floor_area_df (2490, 2)
sdg_df (2490, 19)
education_sdg_df (2482, 102)


In [10]:
print("education_sdg_df",education_sdg_df.shape)
education_sdg_df.to_csv('./data/output/Population_Schools_SDGs_Clean.csv',index=False)

with pd.option_context("display.max_rows",None):
    display(education_sdg_df.head(6).T)

education_sdg_df (2482, 102)


,0,1,2,3,4,5
PSGC_BRGY,PH150702001,PH150702002,PH150702004,PH150702005,PH150702006,PH150702007
both_age_0_to_4,187,459,120,583,727,591
male_age_0_to_4,117,232,63,332,302,308
female_age_0_to_4,70,227,57,251,425,283
both_age_5_to_9,79,161,62,305,354,236
male_age_5_to_9,35,77,40,159,201,114
female_age_5_to_9,44,84,22,146,153,122
both_age_10_to_14,73,133,80,269,398,220
male_age_10_to_14,42,69,39,155,217,121
female_age_10_to_14,31,64,41,114,181,99
